In [1]:
import pandas as pd
from datetime import datetime
from calendar import monthrange

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False) 

In [450]:
df_22 = pd.read_csv('Filtrado 100 Nuevo/MIGTAB022_100.csv',sep=";")
NombreArchivo = ['MIGTAB018_100_Daha','MIGTAB018_100_Lina'\
                 
                 ,'MIGTAB018_100_Julian','OperacionesSandra','MIGTAB018_Fabian'
                 ]

df_18 = [] 
for i in NombreArchivo:
    df = pd.read_csv(f'Arhivos Equipo/Finales/{i}.csv',sep=";")
    df.columns = df.columns.str.strip()
    df_18.append(df)


df_18 = pd.concat(df_18, ignore_index=True)
creditos_df = df_18['op_operacion'].drop_duplicates().to_list()

df_18['op_operacion'].count()

101

In [368]:
#len(creditos_df)

In [452]:
creditos_df = df_18['op_operacion'].drop_duplicates().to_list()


In [453]:
campos = ['op_plazo','op_monto','op_operacion','op_fecha_ini','op_dia_fijo']

In [454]:
df_22_new = df_22[df_22['ts_operacion'].isin(creditos_df)][['ts_operacion','ts_porcentaje']].drop_duplicates().reset_index()

In [455]:
parametros_df = pd.merge(df_22_new,df_18[campos],left_on='ts_operacion', right_on='op_operacion',how="inner")
parametros_df['EfectivoAnual'] = ((1 + (parametros_df['ts_porcentaje'] / 100) / 12) ** 12 - 1) * 100
parametros_df['EfectivoAnual'] = parametros_df['EfectivoAnual'].round(2)

In [456]:
import pandas as pd
from datetime import datetime


rows = []
rows_22 = []
rows_19 = []
rows_20 = []
rows_21 = []


for index, row in parametros_df.iterrows():

    # Parámetros
    dia_corte = row['op_dia_fijo']
    monto = row['op_monto']
    tnominal = row['ts_porcentaje']
    tasa_anual = row['EfectivoAnual']
    plazo_meses = row['op_plazo']
    op_operacion = row['op_operacion']
    tasa_mensual = tasa_anual / 12 / 100
    fecha_inicio_prestamo = pd.to_datetime(row['op_fecha_ini'])
    hoy = pd.to_datetime(datetime.today().date()) + pd.Timedelta(days=1)

    cuota = monto * (tasa_mensual * (1 + tasa_mensual)**plazo_meses) / ((1 + tasa_mensual)**plazo_meses - 1)
    cuota = round(cuota, 2)

    saldo = monto

    rows_19.append(
            {
                'ro_operacion': op_operacion,
                'ro_concepto': 'CAP',
                'ro_signo': '',
                'ro_factor': 0,
                'ro_referencial': '',
                'ro_signo_reajuste': '',
                'ro_factor_reajuste': 0,
                'ro_referencial_reajuste': '',
                'ro_valor':  monto,
                'ro_porcentaje_efa':  0, 
                'ro_garantia': 0,
                'ro_financiado': ''
            }
        )
    rows_19.append(
            {
                'ro_operacion': op_operacion,
                'ro_concepto': 'INT',
                'ro_signo': '',
                'ro_factor': 0,
                'ro_referencial': 'T_FIJA',
                'ro_signo_reajuste': '',
                'ro_factor_reajuste': 0,
                'ro_referencial_reajuste': 'T_FIJA',
                'ro_valor':  0,
                'ro_porcentaje_efa':  tasa_anual, 
                'ro_garantia': 0,
                'ro_financiado': ''
            }
        )
    rows_19.append(
            {
                'ro_operacion': op_operacion,
                'ro_concepto': 'IMO',
                'ro_signo': '',
                'ro_factor': 0,
                'ro_referencial': 'T_USURA',
                'ro_signo_reajuste': '',
                'ro_factor_reajuste': 0,
                'ro_referencial_reajuste': '',
                'ro_valor':  0,
                'ro_porcentaje_efa':  '25.62', 
                'ro_garantia': 0,
                'ro_financiado': ''
            }
        )

    for i in range(plazo_meses):
        fecha = fecha_inicio_prestamo + pd.DateOffset(months=i)
        if i == 0:
            # Primera cuota: mismo mes que el inicio
            fecha_inicio = fecha_inicio_prestamo
            ultimo_dia_mes = monthrange(fecha.year, fecha.month)[1]
            dia_seguro = min(dia_corte, ultimo_dia_mes)
            fecha_fin = pd.Timestamp(year=fecha.year, month=fecha.month, day=dia_seguro)
            if fecha_inicio >= fecha_fin:
                # Si el préstamo fue después del corte, se corre al mes siguiente
                fecha_fin += pd.DateOffset(months=1)
        else:
            # Cuotas siguientes
            ultimo_dia_mes = monthrange(fecha.year, fecha.month)[1]
            dia_seguro = min(dia_corte, ultimo_dia_mes)
            fecha_fin = pd.Timestamp(year=fecha.year, month=fecha.month, day=dia_seguro)
            if fecha.day >= dia_corte:
                fecha_fin += pd.DateOffset(months=1)

            fecha_inicio = fecha_fin - pd.DateOffset(months=1)  # Ajustar el inicio al mes anterior
      
            # Verificamos si la fecha fin es correcta
      #  if fecha_fin.month != fecha.month:  # En caso de que la fecha_fin se haya desajustado
       #     ultimo_dia_mes = monthrange(fecha_fin.year, fecha_fin.month)[1]
        #    fecha_fin = pd.Timestamp(year=fecha_fin.year, month=fecha_fin.month, day=ultimo_dia_mes)


        interes = round(saldo * tasa_mensual, 2)
        amortizacion = round(cuota - interes, 2)
        saldo = round(saldo - amortizacion, 2)
        #estado = 3 if fecha <= hoy else ""
        if hoy > fecha_fin: 
           estado = 3
        elif fecha_inicio <= hoy <= fecha_fin:
           estado = 1
        else: 
           estado = 0   

        rows_21.append({
            'am_operacion':  op_operacion,
            'am_dividendo': i + 1,
            'am_concepto': 'CAP',
            'am_estado': estado, 
            'am_periodo': 0,
            'am_cuota': round(cuota,0),
            'am_gracia': 0,
            'am_pagado': round(amortizacion,0) if estado == 3 else 0, 
            'am_acumulado': 0,
            'am_secuencia': 1

        })
        rows_21.append({
            'am_operacion':  op_operacion,
            'am_dividendo': i + 1,
            'am_concepto': 'INT',
            'am_estado': estado, 
            'am_periodo': 0,
            'am_cuota': round(interes,0),
            'am_gracia': 0,
            'am_pagado': round(interes,0) if estado == 3 else 0, 
            'am_acumulado': 0,
            'am_secuencia': 1

        })

        # Fila para Interés
        rows.append({
            'op_operacion': op_operacion,
            'cuota_no': i + 1,
            'tasa_anual': round(tasa_anual,2),
            'plazo_meses': plazo_meses,
            'mes': fecha,
            'tipo': 'Interés',
            'monto': round(interes,0),
            'saldo_restante': round(saldo + amortizacion,0),
            'estado_pago': estado,
            'cuota_total': round(cuota,0),
            'fecha_inicio': fecha_inicio,
            'fecha_fin': fecha_fin,
            'Dia_Corte':dia_corte
        })

        # Fila para Capital
        rows.append({
            'op_operacion': op_operacion,
            'cuota_no': i + 1,
            'tasa_anual': round(tasa_anual,2),
            'plazo_meses': plazo_meses,
            'mes': fecha,
            'tipo': 'Capital',
            'monto': round(amortizacion,0),
            'saldo_restante': round(saldo,0),
            'estado_pago': estado,
            'cuota_total': round(cuota,0),
            'fecha_inicio': fecha_inicio,
            'fecha_fin': fecha_fin,
            'Dia_Corte':dia_corte
        })

        rows_22.append(
            {
                'op_operacion': op_operacion,
                'ts_dividendo': i + 1,
                'ts_fecha': fecha_inicio,
                'ts_concepto': 'INT',
                'ts_porcentaje':tnominal,
                'ts_secuencial': 1,
                'ts_porcentaje_efa':tasa_anual
            })
        
        if estado == 3: 
            di_fecha_can = fecha_fin.strftime('%Y-%m-%d')
        else: di_fecha_can = ''

        rows_20.append({
              'di_operacion': op_operacion,
              'di_dividendo': i + 1,
              'di_fecha_ini': fecha_inicio,
              'di_fecha_ven': fecha_fin,
              'di_de_capital': 'S',
              'di_de_interes': 'S',
              'di_gracia': 5,
              'di_gracia_disp': 0,
              'di_estado': estado,
              'di_dias_cuota': 30,
              'di_intento': 0,
              'di_prorroga': 'N',
              'di_fecha_can':  di_fecha_can,
              'di_subs_cuota': 0,
              'di_subs_acumulado': 0,
              'di_compra_int_vendedor': 0,

        })
 
# Crear DataFrame
df_doble_fila = pd.DataFrame(rows)
df_19 = pd.DataFrame(rows_19)
df_20 = pd.DataFrame(rows_20)
df_21 = pd.DataFrame(rows_21)
df_22 = pd.DataFrame(rows_22)


df_19['ro_porcentaje_efa'] = df_19['ro_porcentaje_efa'].round(2)

# Acumulado por tipo
df_doble_fila['monto_acumulado_tipo'] = round(df_doble_fila.groupby(['tipo', 'op_operacion'])['monto'].cumsum(),0)
df_21['am_acumulado'] = round(df_21.groupby(['am_concepto', 'am_operacion'])['am_cuota'].cumsum(),0)

# Fecha mayor
df_doble_fila['fecha_mayor'] = df_doble_fila.groupby('op_operacion')['fecha_fin'].transform('max')


print(df_doble_fila['op_operacion'].nunique())

C:\Users\GTC-88\AppData\Local\Temp\ipykernel_38892\3217485545.py:22: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fecha_inicio_prestamo = pd.to_datetime(row['op_fecha_ini'])


100


## Actualización de campos

In [457]:
df_18_update = df_doble_fila[['op_operacion','fecha_mayor','plazo_meses','cuota_total']].drop_duplicates()

df_18['op_ciudad'] = 76001
df_18['op_destino'] = 'D01'
df_18['op_oficial'] = '144'
df_18['op_sector'] = 'S02'
df_18['op_oficial'] = '144'
df_18['op_periodo_cap'] = '1'
df_18['op_periodo_int'] = '1'
df_18['op_oficial'] = '144'
df_18['op_numero_reest'] = '0'
df_18['op_tipo_empresa'] = '0010'
df_18['op_base_tasa_desc'] = 'N'
df_18['op_cuota_ballom'] = '0'
df_18['op_anterior'] = ''
df_18['op_migrada'] = ''
df_18['op_fecha_corte'] = ''


df_18 = df_18.merge(df_18_update, on = 'op_operacion', how='inner')

Col_map = {
    'fecha_mayor':'op_fecha_fin',
    'plazo_meses':'op_plazo',
    'cuota_total':'op_cuota',
}

for col_df1, col_df2 in Col_map.items():
    df_18[col_df2] = df_18[col_df1]
    df_18.drop(columns=col_df1, inplace= True)

ColumnasEliminarDF = ['CREAGE','CRECDLIN','CRENROPR','NUMMIG','CAMPOMIG']

for i in ColumnasEliminarDF:
    df_18.drop(columns=i, inplace= True)


df_18['op_fecha_ult_proceso'] = df_18['op_fecha_ini']
df_18['op_fecha_liq'] = df_18['op_fecha_ini']




In [375]:
#campos = ['op_operacion','op_ciudad','op_fecha_ini','op_fecha_fin','op_fecha_ult_proceso','op_fecha_liq','op_monto','op_monto_aprobado','op_plazo','op_cuota','op_operacion','fecha_mayor','plazo_meses','cuota_total']

In [376]:
#campos_df = ['op_operacion','op_ciudad','op_fecha_ini','op_fecha_fin','op_fecha_ult_proceso','op_fecha_liq','op_monto','op_monto_aprobado','op_plazo','op_cuota']
#df_18[df_18['op_operacion']== 50][campos_df]

In [458]:
df = [df_18,df_19,df_20,df_21,df_22]

for i in range(len(df)):

 df[i] = df[i].applymap(lambda x: f"{x:.2f}".replace('.', ',') if isinstance(x, (float, int)) else x)


C:\Users\GTC-88\AppData\Local\Temp\ipykernel_38892\4164389345.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[i] = df[i].applymap(lambda x: f"{x:.2f}".replace('.', ',') if isinstance(x, (float, int)) else x)
C:\Users\GTC-88\AppData\Local\Temp\ipykernel_38892\4164389345.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[i] = df[i].applymap(lambda x: f"{x:.2f}".replace('.', ',') if isinstance(x, (float, int)) else x)
C:\Users\GTC-88\AppData\Local\Temp\ipykernel_38892\4164389345.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[i] = df[i].applymap(lambda x: f"{x:.2f}".replace('.', ',') if isinstance(x, (float, int)) else x)


In [459]:
fecha_formateada = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")

df_22['op_operacion'] = df_22['op_operacion'].apply(lambda x: str(x) + '0000' if x < 100 else str(x))
df_19['ro_operacion'] = df_19['ro_operacion'].apply(lambda x: str(x) + '0000' if x < 100 else str(x))
df_20['di_operacion'] = df_20['di_operacion'].apply(lambda x: str(x) + '0000' if x < 100 else str(x)) 
df_21['am_operacion'] = df_21['am_operacion'].apply(lambda x: str(x) + '0000' if x < 100 else str(x))
df_18['op_operacion'] = df_18['op_operacion'].apply(lambda x: str(x) + '0000' if x < 100 else str(x))

In [460]:
df_18[df_18['op_operacion'] == '530000']

,op_operacion,op_banco,op_anterior,op_migrada,op_cliente,op_nombre,op_sector,op_toperacion,op_oficina,op_moneda,op_comentario,op_oficial,op_fecha_ini,op_fecha_fin,op_fecha_ult_proceso,op_fecha_liq,op_fecha_reajuste,op_monto,op_monto_aprobado,op_destino,op_lin_credito,op_ciudad,op_estado,op_periodo_reajuste,op_reajuste_especial,op_tipo,op_forma_pago,op_dias_anio,op_tipo_amortizacion,op_cuota_completa,op_tipo_cobro,op_tipo_reduccion,op_aceptar_anticipos,op_precancelacion,op_tipo_aplicacion,op_tplazo,op_plazo,op_tdividendo,op_periodo_cap,op_periodo_int,op_dist_gracia,op_gracia_cap,op_gracia_int,op_dia_fijo,op_cuota,op_evitar_feriados,op_num_renovacion,op_renovacion,op_mes_gracia,op_reajustable,op_dias_clausula,op_divcap_original,op_clausula_aplicada,op_traslado_ingresos,op_periodo_crecimiento,op_tasa_crecimiento,op_direccion,op_opcion_cap,op_tasa_cap,op_dividendo_cap,op_clase,op_origen_fondos,op_calificacion,op_estado_cobranza,op_numero_reest,op_edad,op_tipo_crecimiento,op_base_calculo,op_prd_cobis,op_ref_exterior,op_sujeta_nego,op_dia_habil,op_recalcular_plazo,op_usar_tequivalente,op_fondos_propios,op_nro_red,op_tipo_redondeo,op_sal_pro_pon,op_tipo_empresa,op_validacion,op_fecha_pri_cuot,op_gar_admisible,op_causacion,op_convierte_tasa,op_reestructuracion,op_fecha_ult_causacion,op_fecha_prox_segven,op_suspendio,op_fecha_suspenso,op_honorarios_cobranza,op_banca,op_promocion,op_acepta_ren,op_emprendimiento,op_valor_cat,op_grupo,op_ref_grupal,op_grupal,op_fondeador,op_admin_individual,op_estado_hijas,op_tipo_renovacion,op_tipo_reest,op_fecha_reest,op_fecha_reest_noestandar,op_beneficio_feci,op_beneficio_tinteres,op_base_tasa_desc,op_tir,op_cuota_ballom,op_div_dist_gracia,op_beneficio_empleado,op_fecha_ult_pago,op_fecha_ult_pago_cap,op_fecha_ult_pago_int,op_monto_ult_pago,op_monto_ult_pago_cap,op_monto_ult_pago_int,op_compra_id_externo,op_compra_activa,op_compra_valor,op_compra_devengando,op_vta_comprador,op_vta_fecha,op_vta_fecha_recompra,op_fecha_corte
85,530000,82303800,,,31240492,LEON CORONADO MYRIAM ELCIRA,S02,410,1701,0.0,REGISTRO MIGRADO EL 2025-03-31 11:01:15,144,1/01/2025,2026-05-30,1/01/2025,1/01/2025,NaN,5462771.0,5462771.0,D01,NaN,76001,1,0,S,N,C,360,FRANCESA,N,A,T,S,S,D,M,17,M,1,1,NaN,0,0,30,374236.0,S,NaN,S,0,N,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,0,NaN,A,E,7,NaN,N,N,NaN,NaN,S,NaN,NaN,NaN,0010,NaN,31/07/2023,N,L,NaN,S,30/03/2025,NaN,N,NaN,NaN,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,0,0,0,NaN,23/07/2024,23/07/2024,23/07/2024,352665.0,344456.0,8209.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [462]:
#df_doble_fila.to_csv(f'Arhivos Equipo/21_{NombreArchivo}.csv',encoding='ISO-8859-1',  decimal=',' ,index=False)
df_22.to_csv(f'Arhivos Equipo/Finales/Finales Automatizado/CA_REAJUSTE_DET_MIG_{fecha_formateada}.txt',encoding='ISO-8859-1',  decimal=',' ,index=False, sep="|",header=False)
df_19.to_csv(f'Arhivos Equipo/Finales/Finales Automatizado/CA_RUBRO_OP_MIG_{fecha_formateada}.txt',encoding='ISO-8859-1',  decimal=',' ,index=False, sep="|",header=False)
df_20.to_csv(f'Arhivos Equipo/Finales/Finales Automatizado/CA_DIVIDENDO_MIG_{fecha_formateada}.txt',encoding='ISO-8859-1',  decimal=',' ,index=False, sep="|",header=False)
df_21.to_csv(f'Arhivos Equipo/Finales/Finales Automatizado/CA_AMORTIZACION_MIG_{fecha_formateada}.txt',encoding='ISO-8859-1',  decimal=',' ,index=False, sep="|",header=False)
df_18.to_csv(f'Arhivos Equipo/Finales/Finales Automatizado/CA_OPERACION_MIG_{fecha_formateada}.txt',encoding='ISO-8859-1',  decimal=',' ,index=False, sep="|",header=False)

In [1]:
#pruebas nuevamene hasta que me quede claro como funciona 

In [ ]:
#nueva linea de codigo